## Mini-Project 2: Network Intrusion Detector


#### CSC 180  Intelligent Systems (Fall 2019)
#### Derrek Gass, Alexander Lee, Jimmy Le
#### 10-11-2019

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, show

import collections

import io
import requests
import shutil
import os, json
import csv

from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.feature_extraction.text as sk_text
from sklearn.metrics import confusion_matrix, classification_report


import numpy as np
import pandas as pd
import dask.dataframe as dd

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers, regularizers




# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, collections.Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

    
# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Plot a confusion matrix.
# cm is the confusion matrix, names are the names of the classes.
def plot_confusion_matrix(cm, names, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(names))
    plt.xticks(tick_marks, names, rotation=45)
    plt.yticks(tick_marks, names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    

# Plot an ROC. pred - the predictions, y - the expected output.
def plot_roc(pred,y):
    fpr, tpr, thresholds = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()
    

In [3]:
from dask.distributed import Client, progress
client = Client()

In [4]:
client

Client Scheduler: tcp://127.0.0.1:51114 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.59 GB


In [201]:
#load csv into dask dataframe
#df = dd.read_csv('data/network_intrusion_data.csv')
#load csv into dask dataframe
df = dd.read_csv('data/network_intrusion_onehot.csv')
X = dd.read_csv('data/network_intrusion_normalized.csv')


In [57]:
#load csv into pandas dataframe
df = pd.read_csv('data/network_intrusion_data.csv')

In [41]:
df.columns = [
 'duration',
 'protocol_type',
 'service',
 'flag',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'outcome'
]

In [42]:
df = df.drop_duplicates(subset=None, keep='first', inplace=False)

In [7]:
#remove_outliers(df, "overall_rating", 2.8)

In [36]:
df_abnormal = df.loc[df['outcome'] != 'normal.']

In [37]:
df_abnormal.compute()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome
743,184,tcp,telnet,SF,1511,2957,0,0,0,3,...,3,1.00,0.00,1.00,0.67,0.0,0.00,0.0,0.0,buffer_overflow.
744,305,tcp,telnet,SF,1735,2766,0,0,0,3,...,4,1.00,0.00,0.50,0.50,0.0,0.00,0.0,0.0,buffer_overflow.
4048,79,tcp,telnet,SF,281,1301,0,0,0,2,...,10,1.00,0.00,1.00,0.30,0.0,0.00,0.0,0.1,loadmodule.
4112,25,tcp,telnet,SF,269,2333,0,0,0,0,...,2,0.03,0.06,0.01,0.00,0.0,0.00,0.0,0.0,perl.
7600,0,tcp,telnet,S0,0,0,0,0,0,0,...,6,1.00,0.00,0.20,0.33,1.0,0.83,0.0,0.0,neptune.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68959,0,udp,private,SF,28,0,0,3,0,0,...,96,0.38,0.01,0.38,0.00,0.0,0.00,0.0,0.0,teardrop.
68960,0,udp,private,SF,28,0,0,3,0,0,...,97,0.38,0.01,0.38,0.00,0.0,0.00,0.0,0.0,teardrop.
68961,0,udp,private,SF,28,0,0,3,0,0,...,98,0.38,0.01,0.38,0.00,0.0,0.00,0.0,0.0,teardrop.
68962,0,udp,private,SF,28,0,0,3,0,0,...,99,0.39,0.01,0.39,0.00,0.0,0.00,0.0,0.0,teardrop.


In [38]:
smurf = df_abnormal.loc[df_abnormal['outcome']=='smurf.']

In [39]:
smurf.compute()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome
7792,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,3,0.02,0.02,0.02,0.0,0.0,0.0,0.0,0.0,smurf.
7793,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,13,0.08,0.02,0.08,0.0,0.0,0.0,0.0,0.0,smurf.
7794,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,23,0.14,0.02,0.14,0.0,0.0,0.0,0.0,0.0,smurf.
7795,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,33,0.19,0.02,0.19,0.0,0.0,0.0,0.0,0.0,smurf.
7796,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,43,0.23,0.02,0.23,0.0,0.0,0.0,0.0,0.0,smurf.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65250,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.00,0.00,1.00,0.0,0.0,0.0,0.0,0.0,smurf.
65251,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.00,0.00,1.00,0.0,0.0,0.0,0.0,0.0,smurf.
65255,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.00,0.00,1.00,0.0,0.0,0.0,0.0,0.0,smurf.
65256,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,255,1.00,0.00,1.00,0.0,0.0,0.0,0.0,0.0,smurf.


In [43]:
from dask_ml.preprocessing import Categorizer, DummyEncoder

In [ ]:
df = df.categorize()
df.dtypes

In [ ]:
encoder = DummyEncoder()
df = encoder.fit_transform(df)

In [ ]:
df.compute()

In [48]:
normal = df.loc[df['outcome_normal.']==1]

In [52]:
normal.compute()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,outcome_back.,outcome_imap.,outcome_satan.,outcome_phf.,outcome_nmap.,outcome_multihop.,outcome_warezmaster.,outcome_warezclient.,outcome_spy.,outcome_rootkit.
0,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,235,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,219,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72015,0,310,1881,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
72016,0,282,2286,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
72017,0,203,1200,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
72018,0,291,1200,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [199]:
df.head(20)

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,outcome_imap.,outcome_satan.,outcome_phf.,outcome_nmap.,outcome_multihop.,outcome_warezmaster.,outcome_warezclient.,outcome_spy.,outcome_rootkit.,intruder_status
0,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,235,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,219,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,212,1940,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,0,159,4087,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0,210,151,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,0,212,786,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,210,624,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df['intruder_status']=df['outcome_normal.'].apply(lambda x: 0 if x ==1 else 1, meta=('outcome_normal.', 'int64'))


In [ ]:
df.compute()

In [151]:
encode_csv = X_copy.to_csv(r'data/network_intrusion_normalized.csv', single_file=True, header=True, index=False)

In [26]:
bad = df.loc[df['intruder_status']==1]
bad.compute()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,outcome_imap.,outcome_satan.,outcome_phf.,outcome_nmap.,outcome_multihop.,outcome_warezmaster.,outcome_warezclient.,outcome_spy.,outcome_rootkit.,intruder_status
743,184,1511,2957,0,0,0,3,0,1,2,...,0,0,0,0,0,0,0,0,0,1
744,305,1735,2766,0,0,0,3,0,1,2,...,0,0,0,0,0,0,0,0,0,1
4048,79,281,1301,0,0,0,2,0,1,1,...,0,0,0,0,0,0,0,0,0,1
4112,25,269,2333,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
7600,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68959,0,28,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
68960,0,28,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
68961,0,28,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
68962,0,28,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [27]:
good = df.loc[df['intruder_status']==0]
good.compute()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,outcome_imap.,outcome_satan.,outcome_phf.,outcome_nmap.,outcome_multihop.,outcome_warezmaster.,outcome_warezclient.,outcome_spy.,outcome_rootkit.,intruder_status
0,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,235,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,219,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72015,0,310,1881,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
72016,0,282,2286,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
72017,0,203,1200,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
72018,0,291,1200,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

X = df
y = df['intruder_status']



In [154]:
X = X.drop('intruder_status', axis=1)

In [166]:
X['intruder_status']=y

In [202]:
X.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,outcome_imap.,outcome_satan.,outcome_phf.,outcome_nmap.,outcome_multihop.,outcome_warezmaster.,outcome_warezclient.,outcome_spy.,outcome_rootkit.,intruder_status
0,-0.10785,-0.004261,-0.039036,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.10785,-0.004263,-0.025041,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.10785,-0.004272,-0.025041,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.10785,-0.004273,-0.013612,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,-0.10785,-0.004273,-0.013612,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
X_copy = encode_numeric_zscore(X_copy, "dst_bytes")

In [109]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(scaler.fit(X_copy))
print(scaler.transform(X_copy))

StandardScaler(copy=True, with_mean=True, with_std=True)
[[-0.10785062 -0.00426106 -0.03903586 ... -0.07856036 -0.00370646
  -0.00828813]
 [-0.10785062 -0.00426325 -0.0250414  ... -0.07856036 -0.00370646
  -0.00828813]
 [-0.10785062 -0.00427204 -0.0250414  ... -0.07856036 -0.00370646
  -0.00828813]
 ...
 [-0.10785062 -0.00428083 -0.02729433 ... -0.07856036 -0.00370646
  -0.00828813]
 [-0.10785062 -0.00423249 -0.02729433 ... -0.07856036 -0.00370646
  -0.00828813]
 [-0.10785062 -0.00427204 -0.02673521 ... -0.07856036 -0.00370646
  -0.00828813]]


In [95]:
from dask_ml.model_selection import train_test_split

In [203]:
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [204]:
x_train.shape

(Delayed('int-381eac8c-afa9-4125-84d7-a85318593072'), 142)

In [205]:
y_train.shape

(dd.Scalar<size-ag..., dtype=int64>,)

In [206]:
x_test.shape

(Delayed('int-5f4456e0-cae9-4d99-ac99-e1a46d159779'), 142)

In [207]:
y_test.shape

(dd.Scalar<size-ag..., dtype=int64>,)

In [161]:
from sklearn.ensemble import RandomForestClassifier
from distributed import Executor, progress, wait
e = Executor()
e

/Users/derrekgass/anaconda3/lib/python3.7/site-packages/distributed/client.py:3960: UserWarning: Executor has been renamed to Client
  warnings.warn("Executor has been renamed to Client")
/Users/derrekgass/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:52147 Dashboard: http://127.0.0.1:52148/status,Cluster Workers: 4 Cores: 8 Memory: 8.59 GB


In [208]:
df_train, df_test = train_test_split(X)

In [209]:
columns = df_train.columns.values.tolist()

In [210]:
columns.remove('intruder_status')

In [214]:
est.score(df_test[columns], df_test.intruder_status)

0.9998638066053797

In [226]:
from sklearn.metrics import classification_report
target_names = ['Intruder', 'Normal']
y_pred = model.predict(df_test[columns])
print(classification_report(y_pred, df_test.intruder_status, target_names=target_names))

ValueError: cannot convert float NaN to integer

In [223]:
%time
def predict(est, X):
    return est.predict(X[columns])


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


In [124]:
#!!!!!!!!        OLD          !!!!!!!!!!!!!!!
 
model = Sequential()
model.add(Dense(32, input_dim=141, activation='relu')) # Hidden 1
model.add(Dropout(0.1))
model.add(Dense(16, 
                kernel_regularizer=regularizers.l1(0.01),
                activity_regularizer=regularizers.l2(0.01), activation='relu')) # Hidden 2
model.add(Dense(1)) # Output

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=False)

#model.compile(loss='mean_squared_error', optimizer=adam)
model.compile(loss='mean_squared_error', optimizer=sgd)

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x_train,y_train, validation_data=(x_test, y_test), batch_size=128, callbacks=[monitor, checkpointer], verbose=2,epochs=100)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.
model.load_weights('best_weights.hdf5')

pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred, y_test))
print("Score (RMSE): {}".format(score))
chart_regression(pred.flatten(), y_test.values)

ValueError: Input arrays should have the same number of samples as target arrays. Found nan input samples and 131061 target samples.

In [128]:
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 0.4522813754826082


In [129]:
# print out prediction
df_y = pd.DataFrame(y_test.values, columns=['expected'])
df_pred = pd.DataFrame(pred, columns=['predicted'])

#rounding
df_pred = np.round(df_pred, decimals=2)

result = pd.concat([df_y, df_pred],axis=1)
result['Δ']=np.absolute(df_pred['predicted']-df_y['expected'])
result.head(15)

,expected,predicted,Δ
0,1.5,1.82,0.32
1,3.5,3.90,0.40
2,2.5,3.29,0.79
3,4.5,4.23,0.27
4,4.5,4.89,0.39
5,2.5,2.53,0.03
6,2.5,3.15,0.65
7,1.5,1.66,0.16
8,3.5,3.08,0.42
9,3.0,2.49,0.51


In [130]:
model.summary()

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_120 (Dense)            (None, 32)                64032     
_________________________________________________________________
dropout_36 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_121 (Dense)            (None, 16)                528       
_________________________________________________________________
dense_122 (Dense)            (None, 1)                 17        
Total params: 64,577
Trainable params: 64,577
Non-trainable params: 0
_________________________________________________________________
